# Smart Grid Research: Simulation "Hands-on" Scenario

## This Jupyter Notebook introduces the basic scenario for simulating a very basic scenario in a smart grid. 

Add Python imports.

In [1]:
import import_ipynb

# Only for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

import mosaik.util

Define how to start the adapter.

In [2]:
SIM_CONFIGURATION = {
    'CSV': {
        'python': 'mosaik_csv:CSV',
    },
    'PV': {
        'python': 'mosaik_components.pv.pvsimulator:PVSimulator'
    },
    'Collector': {
        'python': 'simulators.collector:Collector'
    }
}

Define basic constants such as the duration of the simulation.

In [3]:
START = '2014-01-01 00:00:00'
SIMULATION_END = 24 * 3600  # 1 day

Initialize the simulation by defining the "world". 

In [4]:
world = mosaik.World(SIM_CONFIGURATION)


        ____                              _ _
       /    \                            (_) |
  ____/      \  _ __ ___   ___  ___  __ _ _| | __
 /    \      / | '_ ` _ \ / _ \/ __|/ _` | | |/ / 
/      \____/  | | | | | | (_) \__ \ (_| | |   <     
\      /    \  |_| |_| |_|\___/|___/\__,_|_|_|\_\    
 \____/      \____
 /    \      /    \    mosaik version:     3.3.3
/      \____/      \   mosaik API version: 3.0.12
\      /    \      /   Python version:     3.9.5
 \____/      \____/    OS:                 Linux-5.15.0-125-generic-x86_64-with-glibc2.31
      \      /         Documentation:      https://mosaik.readthedocs.io/en/3.3.3/
       \____/          Get in touch:       https://github.com/orgs/OFFIS-mosaik/discussions




Start simulator processes.

In [5]:
# PV
PV_DATA = 'csv_data/pv_10kw.csv'
SIM_START = '2014-01-01 00:00:00'
pv_data = world.start("CSV",
                      sim_start=SIM_START,
                      datafile=PV_DATA)

# Household
HH_DATA = 'csv_data/household_load_profile.csv'
household_data_simulator = world.start("CSV",
                                       sim_start=SIM_START,
                                       datafile=HH_DATA)

2024-12-09 16:34:32.323 | INFO     | mosaik.scenario:start:311 - Starting "CSV" as "CSV-0" ...


RuntimeError: Cannot run the event loop while another loop is running

Instantiate model entities and parametrize.

In [ ]:
pv_data_model = pv_data.PV()

hh_data_model = household_data_simulator.Household()

collector = world.start('Collector')
monitor = collector.Monitor()

Connect models via dataflow. 

In [ ]:
world.connect(pv_data_model, monitor, 'P[W]')
world.connect(hh_data_model, monitor, 'P[W]')

Start the simulation. 

In [ ]:
world.run(until=SIMULATION_END)